In [1]:
import numpy as np
import pandas as pd
import os, sys, json

In [2]:
def process(sign, path_prefix, fun_parse_name, fun_read_one_file):
    signs = []
    sample = []
    ilist = []
    oa = []
    aa = []
    kappa = []
    each_acc = []
    for name in os.listdir(path_prefix): 
        try:
            path = "%s/%s" %(path_prefix, name)
            real_sign, cur_sample, cur_index = fun_parse_name(sign, name)
            signs.append(real_sign)
            sample.append(cur_sample)
            ilist.append(cur_index)
#             print('a', real_sign, 'b', sign)
            temp = fun_read_one_file(path)
            oa.append(temp.get('oa', -1))
            aa.append(temp.get('aa', -1))
            kappa.append(temp.get('kappa', -1))
            each_acc.append(temp.get('each_acc' ''))
        except Exception as e:
            print(e)
            continue

    df = pd.DataFrame({'sign':signs, 'sample': sample, 'ilist': ilist, 'oa':oa, 'aa':aa, 'kappa':kappa, 'each_acc':each_acc})
    return df

In [3]:
# ssgrn
def ssgrn_parse_name(sign, name):
    s, i = [int(x) for x in name.split('.')[0].split('_')]
    return sign, s, i

def ssgrn_read_one_file(path):
    temp = {}
    with open(path, 'r', encoding='utf8') as fin:
        start = False
        each_acc = []
        for line in fin:
            if 'Test set' in line:
                start = True
            if start:
                if 'OA' in line:
                    temp['oa'] = float(line.split('=')[-1])
                if 'AA' in line:
                    temp['aa'] = float(line.split('=')[-1])
                if 'Kappa' in line:
                    temp['kappa'] = float(line.split('=')[-1])
                if 'Class' in line:
                    line = line.strip('.\n')
                    items = line.split(' ')
                    class_index = int(items[0].split('_')[-1].strip(':'))
                    val = float(items[-1])
                    each_acc.append(val)
            temp['each_acc'] = each_acc
    return temp

In [4]:
for sign in ['indian', 'pavia', 'salinas']:
    ssgrn_path_prefix = '../../result/ssgrn/log_%s/' % sign
    print(sign)
    df = process(sign, ssgrn_path_prefix, ssgrn_parse_name, ssgrn_read_one_file)
    df_temp = df[(df['oa']!=-1) &(df['aa']!=-1) &(df['kappa']!=-1)]
    tt = df_temp.groupby(['sample']).mean()
    print(tt)
    print('----------------')

indian
        ilist        oa         aa     kappa
sample                                      
10        5.5  0.776886  87.242651  0.749141
20        5.5  0.876722  93.544222  0.860193
30        5.5  0.905616  95.422785  0.892733
40        5.5  0.924881  96.282505  0.914183
50        5.5  0.945176  97.261924  0.937224
60        5.5  0.945634  97.285147  0.937642
70        5.5  0.949914  97.554407  0.942469
80        5.5  0.953904  97.663054  0.946923
----------------
pavia
        ilist        oa         aa     kappa
sample                                      
10        5.5  0.657459  69.573227  0.571881
20        5.5  0.823756  84.686654  0.773137
30        5.5  0.914452  93.267281  0.888403
40        5.5  0.930062  94.585279  0.908454
50        5.5  0.950782  96.632677  0.935350
60        5.5  0.960704  96.842332  0.948089
70        5.5  0.952852  96.483259  0.937811
80        5.5  0.961977  96.782366  0.950418
----------------
salinas
        ilist        oa         aa     kappa


In [4]:
sign = 'indian'
ssgrn_path_prefix = '../../result/ssgrn/log_%s/' % sign
df = process(sign, ssgrn_path_prefix, ssgrn_parse_name, ssgrn_read_one_file)
sample = 20
df[(df['sign']==sign)&(df['sample']==sample)]

,sign,sample,ilist,oa,aa,kappa,each_acc
10,indian,20,1,0.857315,93.235038,0.838884,"[100.0, 80.7528, 77.1605, 95.8525, 92.8726, 99..."
11,indian,20,10,0.867169,93.103438,0.849136,"[100.0, 78.7642, 71.4815, 100.0, 93.7365, 94.3..."
12,indian,20,2,0.904776,94.757680,0.891830,"[100.0, 79.4034, 85.4321, 95.3917, 92.0086, 97..."
13,indian,20,3,0.886777,94.327631,0.871813,"[100.0, 83.0966, 90.7407, 97.6959, 88.3369, 97..."
14,indian,20,4,0.861840,93.274634,0.843709,"[100.0, 71.5909, 90.3704, 96.3134, 89.4168, 97..."
15,indian,20,5,0.885571,94.233390,0.869870,"[100.0, 72.5142, 85.679, 95.8525, 98.7041, 99...."
16,indian,20,6,0.883861,93.180056,0.868007,"[100.0, 88.9915, 80.0, 98.1567, 79.6976, 93.66..."
17,indian,20,7,0.871795,93.286703,0.855020,"[100.0, 74.3608, 80.8642, 98.1567, 85.3132, 97..."
18,indian,20,8,0.889794,94.278742,0.874994,"[100.0, 79.3324, 86.6667, 99.0783, 87.905, 97...."
19,indian,20,9,0.858321,91.764907,0.838670,"[100.0, 76.0653, 85.9259, 100.0, 87.473, 88.73..."


In [6]:
select_index = 4
path_save = '../../result/ssgrn/ssgrn_select/%s_%s_%s' % (sign, sample, select_index)
us = df[(df['sample']==sample)&(df['ilist']==select_index)]
jj = {
    'oa': us['oa'].values[0],
    'aa':  us['aa'].values[0],
    'kappa':  us['kappa'].values[0],
    'each_acc':  us['each_acc'].values[0]
}
rr = {'eval': jj}
with open(path_save, 'w') as fout:
    fout.write(json.dumps(rr))

In [5]:
# default
def json_read_one_file(path):
    res = {}
    with open(path, 'r', encoding='utf8') as fin:
        temp = json.loads(fin.read())
    ev = temp['eval']
    res['oa'] = ev['oa']
    res['aa'] = ev['aa']
    res['kappa'] = ev['kappa']
    res['each_acc'] = ev['each_acc']
    return res

def json_parse_name(sign, name):
    file_sign = '_'.join(name.split('_')[0:2])
    s,i,_= name.split('_')[-3:]
    if sign == '':
        sign = file_sign
    return sign, s, i


In [8]:
path_prefix = '../../result/res_conv/'
df = process('', path_prefix, json_parse_name, json_read_one_file)
df.groupby(['sign', 'sample']).mean()

C:\Users\chenn\AppData\Local\Temp\ipykernel_1180\748470777.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby(['sign', 'sample']).mean()


oa         aa      kappa
sign                sample                                 
indian_conv1d.json  10      50.060462  60.892639  44.165427
                    20      57.736191  68.595974  52.622986
                    30      64.940337  76.275154  60.442860
                    40      69.301604  78.965652  65.401085
                    50      68.963568  79.171921  64.966068
                    60      72.676714  81.766366  69.089324
                    70      73.450280  80.525149  69.836268
                    80      75.828422  82.635469  72.412472
indian_conv2d.json  10      41.790068  48.856171  34.973426
                    20      50.107657  61.091011  44.260513
                    30      59.224885  71.026398  54.295648
                    40      61.460942  70.188365  56.570879
                    50      59.731993  69.526753  55.015014
                    60      66.862662  75.643869  62.640882
                    70      66.844598  72.653832  62.412733
                    80      66.568435  72.256262  62.106296
pavia_conv1d.json   10      73.575880  80.409041  66.623872
                    20      80.358250  84.712552  74.644099
                    30      82.258975  84.898541  76.985969
                    40      83.187005  87.726561  78.350094
                    50      84.571186  87.665467  79.942867
                    60      79.002273  73.144809  71.739675
                    70      86.569544  84.612763  82.311532
                    80      84.183470  87.126884  79.484735
pavia_conv2d.json   10      67.489106  73.529887  58.791408
                    20      75.978026  80.254346  69.124073
                    30      75.372418  81.590975  68.782137
                    40      79.923142  84.827575  74.241329
                    50      80.714454  85.631957  75.229645
                    60      70.651577  78.878065  63.442095
                    70      75.642291  80.315992  69.139367
                    80      74.348012  84.586237  68.051046
salinas_conv1d.json 10      79.922548  88.461822  77.771518
                    20      87.121114  93.592456  85.685843
                    30      88.802774  94.257825  87.548602
                    40      84.740414  91.334835  83.001476
                    50      89.055111  94.894658  87.824071
                    60      88.391732  94.086550  87.035460
                    70      89.395386  94.614786  88.162149
                    80      89.686844  94.506776  88.446772
salinas_conv2d.json 10      80.578851  87.630839  78.449085
                    20      83.495326  90.981349  81.691265
                    30      85.272419  92.296757  83.671086
                    40      82.587822  89.660954  80.740489
                    50      83.202010  86.433089  81.207299
                    60      83.664918  88.760613  81.790859
                    70      80.692335  86.197773  78.661497
                    80      78.579349  85.338645  76.410434

In [10]:
df[(df['sign']=='salinas_conv1d.json')].sort_values(['sample', 'ilist'])

,sign,sample,ilist,oa,aa,kappa,each_acc
160,salinas_conv1d.json,10,0,80.544387,89.642196,78.500128,[99.44972486 99.46178687 91.35300102 96.315028...
161,salinas_conv1d.json,10,1,80.099687,89.434729,77.991817,[98.94947474 99.46178687 92.37029502 95.881502...
162,salinas_conv1d.json,10,2,77.703867,84.121144,75.234799,[85.04252126 99.78471475 64.24211597 95.520231...
163,salinas_conv1d.json,10,3,80.570327,89.342673,78.476102,[98.7993997 99.46178687 90.89521872 96.026011...
164,salinas_conv1d.json,10,4,80.694473,89.768370,78.654742,[98.94947474 99.40796555 92.16683622 96.098265...
165,salinas_conv1d.json,20,0,85.944730,93.410113,84.411844,[99.59778783 99.67620076 98.92638037 99.708879...
166,salinas_conv1d.json,20,1,87.483506,93.603032,86.071189,[ 99.64806435 99.7301673 99.13087935 99.70...
167,salinas_conv1d.json,20,2,88.015016,93.687787,86.649495,[ 99.59778783 99.64921749 98.87525562 99.70...
168,salinas_conv1d.json,20,3,85.976324,93.441133,84.450953,[ 99.59778783 99.75715057 98.87525562 99.70...
169,salinas_conv1d.json,20,4,88.185991,93.820217,86.845736,[ 99.49723479 99.7301673 98.77300613 99.70...


In [13]:
path_prefix = '../../result/res_transformer//'
df = process('', path_prefix, json_parse_name, json_read_one_file)
# df.groupby(['sign', 'sample']).mean()
df[df['sign']=='salinas_cross'].sort_values(['sign', 'sample'])

,sign,sample,ilist,oa,aa,kappa,each_acc
80,salinas_cross,10,0,94.113287,96.402904,93.466688,[100. 99.97308934 100. 98.04...
81,salinas_cross,10,1,94.298579,97.209976,93.672686,[100. 100. 100. 97.39...
82,salinas_cross,10,2,94.174433,97.235868,93.538829,[100. 100. 100. 98.12...
83,salinas_cross,10,3,95.217625,97.820903,94.686644,[100. 99.89235737 100. 97.90...
84,salinas_cross,10,4,94.165169,97.420346,93.524432,[100. 99.91926803 100. 98.26...
85,salinas_cross,10,4,94.393078,97.222810,93.777604,[100. 99.94617869 100. 97.76...
86,salinas_cross,20,0,96.043413,97.962294,95.591562,[ 99.84917044 100. 100. 100. ...
87,salinas_cross,20,1,95.244290,97.805544,94.706195,[100. 100. 100. 99.70...
88,salinas_cross,20,2,96.242264,98.228656,95.823607,[100. 100. 100. 100. ...
89,salinas_cross,20,3,96.706871,98.275711,96.334435,[ 98.99446958 100. 100. 99.92...


In [20]:
path_prefix = '../../result/ssftt/res_ssftt/'
df = process('', path_prefix, json_parse_name, json_read_one_file)
df.groupby(['sign', 'sample']).mean()
df[df['sign']=='pavia_ssftt.json'].sort_values(['sign', 'sample'])

,sign,sample,ilist,oa,aa,kappa,each_acc
41,pavia_ssftt.json,10,0,85.032563,87.062640,80.612247,[58.02748829 93.07902784 96.50550503 96.758349...
42,pavia_ssftt.json,10,1,85.819707,87.147286,81.499698,[71.72632533 92.47277214 92.77166108 94.106090...
43,pavia_ssftt.json,10,2,89.703884,89.363980,86.470420,[75.04908624 96.78094318 94.44710388 88.932547...
44,pavia_ssftt.json,10,3,89.064330,89.604262,85.682571,[ 79.45929618 94.4417619 92.91527046 90.50...
45,pavia_ssftt.json,10,4,85.976667,88.204538,81.849145,[75.86467301 89.53806535 90.66539014 93.811394...
46,pavia_ssftt.json,20,0,91.625974,93.304477,89.037594,[ 85.6602632 92.22180471 83.69408369 97.93...
47,pavia_ssftt.json,20,1,92.506339,93.631153,90.181232,[88.56451369 93.16120028 83.79028379 97.667542...
48,pavia_ssftt.json,20,2,92.609635,93.345838,90.268811,[ 92.02843745 93.73557357 82.68398268 96.64...
49,pavia_ssftt.json,20,3,91.893605,92.809268,89.363694,[ 86.643473 93.27392775 78.64357864 97.01...
50,pavia_ssftt.json,20,4,93.088553,94.088554,90.919360,[85.72076842 94.79306458 87.87878788 96.911957...


In [59]:
df[(df['sign']=='salinas_ssftt.json')&(df['sample']=='80')]

,sign,sample,ilist,oa,aa,kappa,each_acc
116,salinas_ssftt.json,80,0,98.974437,99.600843,98.856273,[100. 100. 100. 100. ...
117,salinas_ssftt.json,80,1,98.639520,99.474979,98.483097,[100. 100. 100. 99.92...
118,salinas_ssftt.json,80,2,98.908210,99.567142,98.782110,[ 99.89631934 100. 100. 100. ...
119,salinas_ssftt.json,80,3,99.040663,99.622423,98.929983,[ 99.89631934 100. 100. 100. ...
120,salinas_ssftt.json,80,4,98.838199,99.603255,98.704699,[ 99.74079834 100. 100. 100. ...


In [8]:
path_prefix = '../../result/res_diffusion/'
df = process('', path_prefix, json_parse_name, json_read_one_file)
df.groupby(['sign', 'sample']).mean()
df[df['sign']=='salinas_diffusion.json'].sort_values(['sign', 'sample'])

,sign,sample,ilist,oa,aa,kappa,each_acc
64,salinas_diffusion.json,10,0,93.296151,95.592678,92.561180,[100. 98.89666308 99.8982706 93.85...
65,salinas_diffusion.json,10,1,92.384517,95.396739,91.554196,[100. 99.3810549 100. 95.59...
66,salinas_diffusion.json,10,2,91.543293,94.764605,90.622004,[100. 97.5780409 99.8982706 94.94...
68,salinas_diffusion.json,20,0,96.803509,98.329701,96.446033,[ 99.39668175 100. 100. 98.61...
69,salinas_diffusion.json,20,1,97.214221,98.231367,96.901201,[ 99.39668175 99.94603346 100. 97.67...
70,salinas_diffusion.json,20,2,96.840677,98.168730,96.487255,[ 99.39668175 100. 100. 98.54...
71,salinas_diffusion.json,30,0,98.741822,99.244975,98.598379,[100. 100. 100. 99.92...
72,salinas_diffusion.json,30,1,98.618800,99.264270,98.461595,[100. 100. 100. 99.85...
73,salinas_diffusion.json,30,2,98.124849,99.073325,97.912487,[100. 100. 100. 99.63...
74,salinas_diffusion.json,40,0,98.207108,99.228281,98.004292,[100. 99.81009224 100. 99.18...


In [6]:
path_prefix = '../../result/res_minigcn//'
df = process('', path_prefix, json_parse_name, json_read_one_file)


In [7]:
df

,sign,sample,ilist,oa,aa,kappa,each_acc
0,Indian_10,10,0,50.669045,64.096257,44.969477,[ 75. 44.14668547 32.92682927 54.62...
1,Indian_20,20,0,60.519167,70.020052,55.230369,[ 80.76923077 60.9375 39.38271605 69.12...
2,Indian_30,30,0,63.916369,69.090686,59.438623,[12.5 65.87982833 42.625 60.869565...
3,Indian_40,40,0,67.946198,70.254012,63.663994,[16.66666667 55.18731988 47.97468354 68.527918...
4,Indian_50,50,0,67.399497,69.700710,63.118533,[26.08695652 50.87082729 51.15384615 61.497326...
5,Indian_60,60,0,71.906177,76.153798,68.173005,[ 65.2173913 62.28070175 61.68831169 75.70...
6,Indian_70,70,0,68.973310,75.741796,65.048164,[60.86956522 70.25036819 69.86842105 69.461077...
7,Indian_80,80,0,72.877101,78.241410,69.095733,[ 78.26086957 53.70919881 63.6 68.78...
8,Pavia_20,20,0,68.116725,81.161851,61.085107,[ 50.55211012 59.62746256 89.85088985 96.91...
9,Pavia_30,30,0,82.355432,84.037430,76.620127,[ 60.6271777 92.20688544 79.89366844 82.36...
